In [ ]:
# !pip install -r neces.txt
# !pip install mysql-connector-python

### Library

In [1]:
import requests
import os
import json
import pandas as pd
from tqdm import tqdm
# SQL
import mysql.connector
from mysql.connector import errorcode
# Web-Scrapping
from selenium import webdriver
import time
import datetime

# Magic Database
# https://www.mtgjson.com/api/v5/AllPrintings.json.xz


In [2]:
def sql(_query):
    return pd.read_sql_query(_query, db)

### Create DataBase 

In [3]:
try:
    db = mysql.connector.connect(
        host = "localhost",
        user="root",
        password="1234",
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Acceso denegado: Usuario o contraseña incorrectos")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)
        
cursor = db.cursor()

In [4]:
cursor.execute("CREATE DATABASE IF NOT EXISTS mtg")
db.commit()

### Data Adquisition - SQL

In [ ]:
# Descarga el archivo SQL desde la URL
url_database = "https://mtgjson.com/api/v5/AllPrintings.sql"
response = requests.get(url_database)

if response.status_code == 200:
    # Guarda el contenido descargado en un archivo local
    with open("MTG_Database.sql", "wb") as sql_file:
        sql_file.write(response.content)
else:
    print(f"Error al descargar el archivo SQL. Código de estado: {response.status_code}")

In [5]:
# Configura la conexión a la base de datos MySQL
try:
    db = mysql.connector.connect(
        host = "localhost",
        user="root",
        password="1234",
        database= "mtg",
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Acceso denegado: Usuario o contraseña incorrectos")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)
        
cursor = db.cursor()

In [6]:
# Lee el archivo SQL descargado
try:
    with open('MTG_Database.sql', 'r', encoding='utf-8') as sql_file:
        sql_script = sql_file.read()

    # Divide el script en comandos SQL (separados por ';')
    sql_commands = sql_script.split(';')
    
     # Inicializa la barra de progreso
    total_commands = len(sql_commands)
    with tqdm(total=total_commands, desc="Progreso") as pbar:
        
        # Contar lineas de errores
        lineas_con_errores = []
        current_line = 1
        
        # Itera a través de los comandos SQL
        for command in sql_commands:
            if command.strip():  # Ignora líneas en blanco
                try:
                    # Ejecuta el comando SQL
                    cursor.execute(command)
                    db.commit()
                except mysql.connector.Error as err:
                    lineas_con_errores.append(current_line)
                    db.rollback()
                    
            pbar.update(1)
            current_line += 1 
            
except FileNotFoundError as e:
    print(f"Error al abrir el archivo SQL: {e}")
except mysql.connector.Error as err:
    print(f"Error al conectar a la base de datos: {err}")

# Cierra el cursor y la conexión
cursor.close()
db.close()

# Imprime las líneas con errores al final
print("\nLíneas con errores:")
len(lineas_con_errores)

Progreso: 100%|██████████| 1054838/1054838 [47:39<00:00, 368.85it/s]


Líneas con errores:
[990, 991, 1289, 1290, 1291, 1292, 1303, 1304, 1312, 1313, 1316, 1317, 1319, 1320, 1339, 1340, 1347, 1348, 1353, 1354, 1356, 1357, 1358, 1359, 1366, 1367, 1372, 1373, 1399, 1400, 1401, 1402, 1404, 1405, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1422, 1423, 1424, 1425, 1438, 1439, 1456, 1457, 1464, 1465, 1482, 1483, 1486, 1487, 1492, 1493, 1498, 1499, 1517, 1518, 1522, 1523, 1528, 1529, 1542, 1543, 1550, 1551, 1554, 1555, 1570, 1571, 1807, 1808, 2092, 2093, 2212, 2213, 2506, 2507, 2593, 2594, 2595, 2596, 2661, 2662, 2690, 2691, 2701, 2702, 2709, 2710, 2739, 2740, 2767, 2768, 2796, 2797, 2899, 2900, 2901, 2902, 2967, 2968, 2996, 2997, 3007, 3008, 3015, 3016, 3045, 3046, 3073, 3074, 3102, 3103, 3205, 3206, 3207, 3208, 3227, 3228, 3231, 3232, 3234, 3235, 3238, 3239, 3257, 3258, 3266, 3267, 3268, 3269, 3273, 3274, 3275, 3276, 3283, 3284, 3289, 3290, 3317, 3318, 3319, 3320, 3329, 3330, 3331, 3337, 3338, 3342, 3343, 3352, 3353, 3363, 3364, 3368, 3369, 3373, 3374, 3398, 33

In [9]:
len(lineas_con_errores)

13733

### Exploration DataBase (names-columns)

In [11]:
# Configura la conexión a la base de datos MySQL
try:
    db = mysql.connector.connect(
        host = "localhost",
        user="root",
        password="1234",
        database= "mtg",
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Acceso denegado: Usuario o contraseña incorrectos")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)
        
cursor = db.cursor()

In [8]:
# Obtiene el nombre de todas las tablas en la base de datos "mtg"
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    print(f"Tabla: {table_name}")

    # Obtiene el nombre de todas las columnas de la tabla actual
    cursor.execute(f"DESCRIBE {table_name}")
    columns = cursor.fetchall()
    
    for column in columns:
        column_name = column[0]
        print(f"  Columna: {column_name}")

# Cierra el cursor y la conexión
cursor.close()
db.close()

Tabla: cardforeigndata
  Columna: id
  Columna: faceName
  Columna: flavorText
  Columna: language
  Columna: multiverseId
  Columna: name
  Columna: text
  Columna: type
  Columna: uuid
Tabla: cardidentifiers
  Columna: id
  Columna: cardKingdomEtchedId
  Columna: cardKingdomFoilId
  Columna: cardKingdomId
  Columna: mcmId
  Columna: mcmMetaId
  Columna: mtgArenaId
  Columna: mtgjsonFoilVersionId
  Columna: mtgjsonNonFoilVersionId
  Columna: mtgjsonV4Id
  Columna: mtgoFoilId
  Columna: mtgoId
  Columna: multiverseId
  Columna: scryfallId
  Columna: scryfallIllustrationId
  Columna: scryfallOracleId
  Columna: tcgplayerEtchedProductId
  Columna: tcgplayerProductId
  Columna: uuid
Tabla: cardlegalities
  Columna: id
  Columna: alchemy
  Columna: brawl
  Columna: commander
  Columna: duel
  Columna: explorer
  Columna: future
  Columna: gladiator
  Columna: historic
  Columna: historicbrawl
  Columna: legacy
  Columna: modern
  Columna: oathbreaker
  Columna: oldschool
  Columna: pauper


In [21]:
sql("""
SELECT name, code FROM sets WHERE name LIKE '%ravnica%'
""")


C:\Users\macka\AppData\Local\Temp\ipykernel_13808\3379854766.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(_query, db)


,name,code
0,Guilds of Ravnica,GRN
1,Guilds of Ravnica Promos,PGRN
2,Ravnica: City of Guilds Promos,PRAV
3,Ravnica Allegiance Promos,PRNA
4,Return to Ravnica Promos,PRTR
5,RNA Ravnica Weekend,PRW2
6,GRN Ravnica Weekend,PRWK
7,Ravnica: City of Guilds,RAV
8,Ravnica Allegiance,RNA
9,Return to Ravnica,RTR


In [29]:
df_cards=sql("""
SELECT DISTINCT cards.name AS NAME_CARD, sets.name AS NAME_SET
FROM cards
INNER JOIN sets ON cards.setCode = sets.code;
""")


C:\Users\macka\AppData\Local\Temp\ipykernel_13808\3379854766.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(_query, db)


In [33]:
df_cards.iloc[0, 0]

"Ancestor's Chosen"

In [34]:
df_cards.iloc[0, 1]

'Tenth Edition'

### Scrap Price (https://www.mtgprice.com/sets/Ravnica_Allegiance/Breeding_Pool)

In [3]:
# Configura el controlador de Chrome
driver = webdriver.Chrome()
driver.get(f'https://www.mtgprice.com/sets/Ravnica_Allegiance/Breeding_Pool')
prices=driver.execute_script('return results')
    

In [ ]:
prices[4]

## Cambiar de UNIX a UTF

In [ ]:
timestamp  = 1563092869548
timestamp = timestamp / 1000 # 2019-07-14 08:27:49.548000
date = datetime.datetime.utcfromtimestamp(timestamp)